In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [3]:
df = spark.read.csv('/FileStore/tables/sales_info.csv',inferSchema=True,header=True)

In [4]:
df.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [5]:
df.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [6]:
df.groupBy('Company')

Out[ 11 ]: <pyspark.sql.group.GroupedData at 0x7f30b6872c18>

In [7]:
df.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [8]:
df.groupBy('Company').count().show()

+-------+-----+
Company|count|
+-------+-----+
 APPL| 4|
 GOOG| 3|
 FB| 2|
 MSFT| 3|
+-------+-----+

In [9]:
# If you don't care about doing the aggregation per "group"
df.agg({'Sales':'max'}).show()

+----------+
max(Sales)|
+----------+
 870.0|
+----------+

In [10]:
gdf = df.groupBy('Company')

In [11]:
# agg() and dict syntax for the GroupedData object as well
gdf.agg({'Sales':'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [12]:
gdf.count()

Out[ 24 ]: DataFrame[Company: string, count: bigint]

In [13]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [14]:
df.select(countDistinct('Sales')).show()

+---------------------+
count(DISTINCT Sales)|
+---------------------+
 11|
+---------------------+

In [15]:
df.select(avg('Sales').alias('avg_sales')).show()

+-----------------+
 avg_sales|
+-----------------+
360.5833333333333|
+-----------------+

In [16]:
df.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [17]:
# Format output to reduce sigfigs
from pyspark.sql.functions import format_number

In [18]:
sales_stddev = df.select(stddev('Sales').alias('stddev'))

In [19]:
# format_number() takes colName and a number of decimal places
sales_stddev.select(format_number('stddev',4)).show()

+------------------------+
format_number(stddev, 4)|
+------------------------+
 250.0874|
+------------------------+

In [20]:
# orderBy func is asc by default
df.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [21]:
# to orderBy desc, you have to call out the actual column (object type) and apply .desc() to it
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2993221611493256> in <module> () 
 ----> 1 df . row 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getattr__ (self, name) 
 1323 if name not in self . columns : 
 1324 raise AttributeError(
 -> 1325 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1326 jc = self . _jdf . apply ( name ) 
 1327 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'row'